In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import numpy as np
from datetime import datetime
# Function to extract Product BarCode
def get_barcode(soup):

    try:
        # Outer Tag Object
        barcode = soup.find("span", attrs={"id":"barcode"}) 
        
        # Inner NavigatableString Object
        barcode_value = barcode.text

        # Title as a string value
        barcode_string = barcode_value.strip()

    except AttributeError:
        barcode_string = ""

    return barcode_string

# Function to extract Product Brand
def get_brand(soup):

    try:
        brand = soup.find("span", attrs={"id":"field_brands_value"}).text.strip()

    except AttributeError:

        try:
            # If there is some deal price
            brand = soup.find("span", attrs={"id":"field_brands_value"}).text.strip()

        except:
            brand = ""

    return brand

# Function to extract Product name
def get_product(soup):

    try:
        p = soup.find("h1", attrs={"class":"title-3"})
        product = p.text.strip()
    
#     except AttributeError:
#         try:
#             product = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
    except:
            product = ""

    return product

# Function to extract Category
def get_category(soup):
    try:
        category = soup.find("span", attrs={"id":"field_categories_value"}).text.strip().split(",")[:2]
        category = ", ".join(category)
    except AttributeError:
        category = ""	

    return category

# Function to extract Availability Status
def get_subcategory(soup):
    try:
        subcategory = soup.find("span", attrs={"id":"field_categories_value"}).text.strip().split(",")[-2:]
        subcategory = ", ".join(subcategory)
    except AttributeError:
        subcategory = ""	

    return subcategory

def get_ingredients(soup):
    try:
        ingredients = soup.find("div", attrs={"class":"panel_text"}).text.strip()

    except AttributeError:
        ingredients = ""

    return ingredients

def get_allergens(soup):
    try:
        allergens = soup.find("span", attrs={"class":"allergen"}).text.strip()
#         allergens = available.find("span").string.strip()

    except AttributeError:
        allergens = ""

    return allergens

def get_nova3(soup):
    try:
        s = soup.find("h4", attrs={"class":None}).text.strip()
        if s == "Processed foods" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova3 = ", ".join(nova)
            return nova3
        else:
            return np.nan
    except Exception as e:
        print("Exception")
        
def get_nova4(soup):
    try:
        s = soup.find("h4", attrs={"class":None}).text.strip()
        if s == "Ultra processed foods" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova4 = ", ".join(nova)
            return nova4

        else:
            return np.nan
    except Exception as e:
        print("Exception")
        
def get_nova(soup):
    try:
        s = soup.find("h4", attrs={"class":None}).text.strip()
        if s == "Ultra processed foods" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova4 = ", ".join(nova)
            return s

        elif s == "Processed foods" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova3 = ", ".join(nova)
            return s
        
        elif s == "Processed culinary ingredients" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova2 = ", ".join(nova)
            return s
        
        elif s == "Unprocessed or minimally processed foods" :
            elements = soup.find("div", attrs={"class":"content panel_content"}).find_all('li')
            nova = []
            for element in elements: 
                if ":" in element.text  :
                    nova.append(element.text)
            nova1 = ", ".join(nova)
            return s
        else:
            return s
    except Exception as e:
        print("Exception")

def time_difference(start,end):
    time_diff = end-start
    days = time_diff.days
    seconds = time_diff.seconds
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"Time taken to Complete:{hours} hours, {minutes} minutes, {seconds} seconds."

In [ ]:
start = datetime.now()
start_page = int(input("Enter Page to start from: "))
end_page = int(input("Enter Page to end: "))
all_products = []
for page in range(start_page,end_page+1):
    URL = f'https://world.openfoodfacts.org/{page}'
    HEADERS = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36', 'Accept-Language': 'en-US, en;q=0.5'})
    webpage = requests.get(URL, headers = HEADERS)
    
    # webpage.content
    soup = BeautifulSoup(webpage.content, "html.parser")
    links = soup.find_all("a", attrs={"class": None})
    
    href_lst = []
    for i in links:
        href_lst.append(i.get('href'))
    for p in href_lst:
        if (p is not None) and (re.match("/product/(\d+)/([\w-]+)",p)is not None):
            all_products.append(p)
    print(f"Page {page} links exctracted.")
print("All Links Generated.")
df = { "BRAND NAME":[], "PRODUCT NAME":[], "CATEGORY":[],"SUBCATEGORY":[],"INGREDIENTS":[],"NOVA Group":[],}
#       "NOVA 3":[], "NOVA 4":[]}
print(f"Products to be added: {len(all_products)}")
c= 0
# Loop for extracting product details from each link 
for link in all_products:
    new_webpage = requests.get("https://in.openfoodfacts.org" + link, headers=HEADERS)

    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    # Function calls to display all necessary product information
#     df['BARCODE'].append(get_barcode(new_soup))
    df['BRAND NAME'].append(get_brand(new_soup))
    df['PRODUCT NAME'].append(get_product(new_soup))
    df['CATEGORY'].append(get_category(new_soup))
    df['SUBCATEGORY'].append(get_subcategory(new_soup))
    df['INGREDIENTS'].append(get_ingredients(new_soup))
    df['NOVA Group'].append(get_nova(new_soup))
#     df['NOVA 3'].append(get_nova3(new_soup))
#     df['NOVA 4'].append(get_nova4(new_soup))
    c += 1
    print(f"Product {c} Added.")
print("All items Added.")

nova_df = pd.DataFrame.from_dict(df)
nova_df['BRAND NAME'].replace('', np.nan, inplace=True)
nova_df = nova_df.dropna(subset=['BRAND NAME'])

# writer = pd.ExcelWriter("OpenFoodFacts.xlsx", engine='xlsxwriter')
# nova_df.to_excel("OpenFoodFactsNOVA.xlsx",sheet_name = "SOLID FOOD", header=True, index=False)
# end = datetime.now()
# print(time_difference(start,end))
# print("Excel Ready.")


Enter Page to start from: 0
Enter Page to end: 69
Page 0 links exctracted.
Page 1 links exctracted.
Page 2 links exctracted.
Page 3 links exctracted.
Page 4 links exctracted.
Page 5 links exctracted.
Page 6 links exctracted.
Page 7 links exctracted.
Page 8 links exctracted.
Page 9 links exctracted.
Page 10 links exctracted.
Page 11 links exctracted.
Page 12 links exctracted.
Page 13 links exctracted.
Page 14 links exctracted.
Page 15 links exctracted.
Page 16 links exctracted.
Page 17 links exctracted.
Page 18 links exctracted.
Page 19 links exctracted.
Page 20 links exctracted.
Page 21 links exctracted.
Page 22 links exctracted.
Page 23 links exctracted.
Page 24 links exctracted.
Page 25 links exctracted.
Page 26 links exctracted.
Page 27 links exctracted.
Page 28 links exctracted.
Page 29 links exctracted.
Page 30 links exctracted.
Page 31 links exctracted.
Page 32 links exctracted.
Page 33 links exctracted.
Page 34 links exctracted.
Page 35 links exctracted.
Page 36 links exctracted

In [ ]:
# writer = pd.ExcelWriter("OpenFoodFacts.xlsx", engine='xlsxwriter')
file_name = f"OpenFoodFactsNOVA_{start_page}-{end_page}.csv" 
nova_df.to_csv(file_name,sheet_name = "SOLID FOOD", header=True, index=False)
end = datetime.now()
print(time_difference(start,end))
print("Excel Ready.")


In [1]:
nova_df

NameError: name 'nova_df' is not defined

In [2]:
time_difference(start,end)

NameError: name 'time_difference' is not defined

In [3]:
# wget https://openfoodfacts-images.s3.eu-west-3.amazonaws.com/data/401/235/911/4303/1.jpg